### Calibrate DHI Southern North Sea WaterBench
[WaterBench SoutherNorthSea](https://github.com/DHI/WaterBench-MIKE21HD-SouthernNorthSea) \
The notebook showcases the automatic calibration of the Manning number by minimizing the RMSE of the water levels. 

##### Import necessary modules and define necessary paths

In [1]:
import optuna
import modelskill as ms
import mikeio
from pathlib import Path
import subprocess
import numpy as np
from helpers import run_simulation, read_num_timesteps, Collector, find_zones, suggest_new_manning, create_new_manning_file, create_new_simfile

SIMFILE = Path(r"..\data\model\SNS_Autocal.m21fm")
MANNING_FILE = Path(r"..\data\input\ManningM.dfsu")
BASE_SIM_PATH = Path(r"..\data\model\SNS_Autocal.m21fm - Result Files")
BASE_OBS_PATH = Path(r"..\data\observations")

c:\Users\FSCH\OneDrive - DHI\Workspace\Code\Python\projects\sns_autocalibration\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### Define MIKE Engine path. 
Should be adatped based on the location of the MIKE Engine.

In [2]:
M21_ENGINE = Path(r"C:\Program Files (x86)\DHI\MIKE Zero\2024\bin\x64\FemEngineHD.exe")

##### Create study and Database
- `direction`: Defines if the objective function is minimized or maximized. In this case it will be minimized as the objective function will be the RMSE of the water level.
- `sampler`: The sampling algorithm that will be used to optimize the objective function. In this case the Gaussian process sampler is used to suggest new parameter values as it has been shown to be effective for this model (see [link to paper](https://2025.iahr.org/Home/Submissions)).
- `study_name`: The name of the study
- `storage`: The database is stored in the current directory

In [3]:
study_name = "SNS-Autocalibration_example"
storage = f"sqlite:///{study_name}.db"

study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.GPSampler(seed=0),
    study_name=study_name,
    storage = storage,
    load_if_exists=False,
)

C:\Users\FSCH\AppData\Local\Temp\ipykernel_38868\1831883161.py:6: ExperimentalWarning: GPSampler is experimental (supported from v3.6.0). The interface can change in the future.
  sampler=optuna.samplers.GPSampler(seed=0),
[I 2025-07-02 11:51:45,396] A new study created in RDB with name: SNS-Autocalibration_example


##### Define the paths of the satellite observation files. 
Those are satellite altimetry tracks that have been converted to .dfs0 files with x,y and z columns and a time dimension

In [4]:
observations = []
observations.append(BASE_OBS_PATH / "Altimetry_wl_3a.dfs0")
observations.append(BASE_OBS_PATH / "Altimetry_wl_3b.dfs0")
observations.append(BASE_OBS_PATH / "Altimetry_wl_6a.dfs0")

##### Define collector 
It stores information about the current simulation file and the manning file. The Collector gets updated once a new simulation is run.

In [5]:
collector = Collector(simfile = SIMFILE, 
    manning_file = MANNING_FILE, 
    zones = find_zones(MANNING_FILE))

##### Define trial
Everything that happens within a trial is defined in the "objective" function below. It consists of the following steps:
- Suggest new manning values
- Create new manning file with the suggested manning values
- Create new simulation file referencing the new manning file
- Run the simulation
- Calculate error of each satellite track with modelskill
- Aggregate errors

In [6]:
def objective(trial: optuna.Trial, collector: Collector):


    new_manning_values = suggest_new_manning(trial = trial, 
                                            zones = collector.zones)
    

    new_manning_file = create_new_manning_file(trial_no = trial.number,
                                                manning_file = collector.manning_file, 
                                                zones = collector.zones,
                                                new_values = new_manning_values)

    new_simfile = create_new_simfile(trial_no = trial.number, 
                                    simfile = collector.simfile, 
                                    manning_file = new_manning_file)  

    results_file = Path(Path(f"{Path(new_simfile)} - Result Files") / "Area.dfsu")


    command = f'"{M21_ENGINE}" "{new_simfile}" -mpi 12 -x'
    run_simulation(command = command, timesteps = read_num_timesteps(new_simfile))

    ## Update collector 

    collector.manning_file = new_manning_file
    collector.simfile = new_simfile

    ## Calculate error of each satellite track with modelskill (RMSE)

    cc = None
    for observation in observations:
        obs = ms.TrackObservation(data=mikeio.read(observation, item=[0,1,2]), name=observation.stem)
        sim = ms.model_result(data=mikeio.read(results_file, items=0), name=observation.name)
        matched = ms.match(obs, sim)
        cc = matched if cc is None else cc + matched
    
    individual_errors = cc.skill().reset_index()["rmse"]
    
    ## Aggregate errors (mean)

    aggregated_error = np.mean(individual_errors)

    return aggregated_error

##### Optimize 

In [ ]:
study.optimize(lambda trial: objective(trial, collector), n_trials=10)

Processing:   0%|          | 0/500 [00:00<?, ?step/s]